In [15]:
from typing import Tuple

from pydantic import BaseModel
from dotenv import load_dotenv
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

from third_parties.linkedin import scrape_linkedin_profile
from agents.linkedin_lookup_agent import lookup as linkedin_lookup_agent
from output_parsers import summary_parser, Summary

In [16]:

# Define your Summary model correctly
class Summary(BaseModel):
    summary: str
    facts: list[str]

In [17]:
def parse_summary_output(output: str) -> Summary:
    try:
        lines = output.strip().split('\n')
        summary_text = lines[0].split(":")[1].strip()
        facts = [line.split(":")[1].strip() for line in lines[1:] if line.startswith("Fact")]
        return Summary(summary=summary_text, facts=facts)
    except IndexError:
        print(f"Error parsing output: {output}")
        return Summary(summary="Could not parse summary", facts=[]) # Return a default object

In [18]:
def ice_break_with(name: str) -> Tuple[Summary, str]:
    linkedin_username = linkedin_lookup_agent(name=name)
    linkedin_data = scrape_linkedin_profile(
        linkedin_profile_url=linkedin_username, mock=True
    )

    summary_template = """
    given the information about a person from linkedin {information},
    I want you to create:
    Summary: <short summary here>
    Fact 1: <first interesting fact>
    Fact 2: <second interesting fact>
    """

    summary_prompt_template = PromptTemplate(
        input_variables=["information"],
        template=summary_template,
    )

    llm = ChatOpenAI(temperature=0, model_name="gpt-4") # Use a full model for best results

    chain = LLMChain(llm=llm, prompt=summary_prompt_template) # Use LLMChain

    llm_output = chain.run(information=linkedin_data)
    res = parse_summary_output(llm_output)

    return res, linkedin_data.get("profile_pic_url")

In [19]:
if __name__ == "__main__":
    load_dotenv()

    print("Ice Breaker Enter")
    ice_break_with(name="Harrison Chase")

Ice Breaker Enter


/home/martin/.pyenv/versions/icebreaker/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
I need to find the Linkedin profile page for Harrison Chase.
Action: Crawl Google 4 linkedin profile page
Action Input: Harrison Chasehttps://soaps.sheknows.com/general-hospital/characters/harrison-chase/This is not the correct Linkedin profile page, I need to try again.
Action: Crawl Google 4 linkedin profile page
Action Input: Harrison Chase Linkedinhttps://www.linkedin.com/in/harrison-chaseI have found the correct Linkedin profile page for Harrison Chase.
Final Answer: https://www.linkedin.com/in/harrison-chase

> Finished chain.


/tmp/ipykernel_4306/1503796156.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=summary_prompt_template) # Use LLMChain
/tmp/ipykernel_4306/1503796156.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_output = chain.run(information=linkedin_data)
